In [1]:
import re
import pandas as pd
from collections import Counter

In [7]:
# 測試資料
test_news = '12年前爆發的花蓮縣壽豐鄉鯉魚潭風景區的「綠湖國際大飯店」開發弊案 ，業者王桂霜為讓計畫案順利進行，以250萬元行賄時任內政部都市計畫委員會委員李威儀與其學生藍秀琪；經法院12年審理，2017年12月花蓮高分院更三審判李威儀12年徒刑、藍秀琪9年，皆褫奪公權6年，王桂霜1年9月、褫奪公權1年；最高法院日前駁回上訴，全案全部定讞。此開發計畫在花蓮縣都市計畫委員會審查時，曾5度闖關未過，1999年起死回生，業者雙聯投資實業公司董事長王桂霜，申請將鯉魚潭的22公頃公園等用地，變更為旅館用地與停車場，隔年8月花蓮縣政府通過變更計畫案，函送內政部都市計畫委員會審議。請繼續往下閱讀...都委會專案小組赴現場勘查後，前3次專案會議都明確表達反對之意，強烈質疑公園綠地面積只占6.4%，未達10%的最低法定下限，更何況是風景特定區，認為不宜輕率將綠地變更為建地。王桂霜為求解套，透過花蓮縣政府承辦技士居中介紹，與負責審查此案的的時任內政部都委會專案小組召集人、台科大建築系副教授李威儀搭上線，相約在第4次專案會議的前1天，於台北市西華飯店碰面。李威儀在西華當面向王桂霜提議，可利用他的碩士在職專班學生藍秀琪當「白手套」，由藍秀琪開設的公司來負責規劃開發計畫書，李向王索賄250萬元，分3期付清。李與藍虛構「綠湖觀光飯店規劃案」，試圖以規劃設計費的名義讓賄款合法化，李並教唆都委會2名承辦人登載不實資料，企圖矇混過關。幾經運作，李威儀趕在他的委員任期屆滿之前，於2001年5月、6月連開最後2次專案小組會議，做成「尊重花蓮縣府意見，建議本案原則同意變更」的審查意見，送內政部審議，並在卸任後列席委員會報告，企圖說服與會人士支持。調查局東機站事後獲報，報告花蓮地檢署指揮偵辦，2007年起訴李嫌等人。案經法院審理12年，李威儀等人一度獲判無罪，但被撤銷發回，更三審依違背職務收賄罪，判李、藍各12年、9年徒刑，王桂霜以行賄罪判刑1年9月。最高法院認定更三審判決沒有違誤， 駁回上訴確定。'
name_list = ['王', '藍秀琪', '李', '藍', '王桂霜', '李威儀','李嫌']

In [10]:
def create_sentance_list(test_news:str, name_list:list) -> list:
    """@test_news: 原文
       @name_list: ner抓出的人名
       @return: sentance_list, 用於下一階段
    """
    sentance_list = []
    # 原人名list排序 不知道為啥
    name_list.sort(reverse=True)

    # 把人名從list提出
    for name in name_list: 
        print(name)
        #建立tmp list, 刪掉除了自己
        tmp_name_list = name_list.copy()
        tmp_name_list.remove(name)
        print(tmp_name_list)
        # 切分句子, 並將自己以外的人取代為 "其他人名"
        test_split_content = test_news
        test_split_content = re.sub('|'.join(tmp_name_list), '其他人名', test_split_content)
        test_split_content = test_split_content.replace('。','=。').replace('，','*，').replace('？','+？')
        test_split_content = re.split('，|。|？', test_split_content)
        test_split_content = list(map(lambda x: x.replace('=','。').replace('*','，').replace('+','？'), test_split_content))
        test_split_content = list(filter(None, test_split_content))
        # 建立sentances list, 即上拼接下文
        for i, s in enumerate(test_split_content):
            sentance = ''
            if name in s:
                # 正常有前後文共3句的情況
                # 第1句出現句號 -> 拼2+3
                # 第2句出現句號 -> 拼1+2
                # 第3句或沒有句號 -> 拚1+2+3
                try:
                    start = test_split_content[i-1]
                    mid = test_split_content[i]
                    end = test_split_content[i+1]
                    if start.find('。') > 0:
                        sentance = mid + end
                    elif mid.find('。') > 0:
                        sentance = start + mid
                    else:
                        sentance = start + mid + end
                except:
                    # 若名字出現在頭尾的例外處理
                    try:             
                        start = test_split_content[i-1]
                        mid = test_split_content[i]
                        if start.find('。') > 0:
                            sentance = mid 
                        else:
                            sentance = start + mid 
                    except:               
                        mid = test_split_content[i]
                        end = test_split_content[i+1]
                        if mid.find('。') > 0:
                            sentance = mid
                        else:
                            sentance = mid + end
                sentance_list.append((sentance, name))
                
    return sentance_list

In [11]:
sentance_list = create_sentance_list(test_news, name_list)
sentance_list

藍秀琪
['藍', '王桂霜', '王', '李嫌', '李威儀', '李']
藍
['藍秀琪', '王桂霜', '王', '李嫌', '李威儀', '李']
王桂霜
['藍秀琪', '藍', '王', '李嫌', '李威儀', '李']
王
['藍秀琪', '藍', '王桂霜', '李嫌', '李威儀', '李']
李嫌
['藍秀琪', '藍', '王桂霜', '王', '李威儀', '李']
李威儀
['藍秀琪', '藍', '王桂霜', '王', '李嫌', '李']
李
['藍秀琪', '藍', '王桂霜', '王', '李嫌', '李威儀']


[('其他人名與藍虛構「綠湖觀光飯店規劃案」，試圖以規劃設計費的名義讓賄款合法化，', '藍'),
 ('更三審依違背職務收賄罪，判其他人名、藍各12年、9年徒刑，其他人名以行賄罪判刑1年9月。', '藍'),
 ('由其他人名開設的公司來負責規劃開發計畫書，其他人名向王索賄250萬元，分3期付清。', '王'),
 ('由其他人名開設的公司來負責規劃開發計畫書，李向其他人名索賄250萬元，分3期付清。', '李'),
 ('李與其他人名虛構「綠湖觀光飯店規劃案」，試圖以規劃設計費的名義讓賄款合法化，', '李'),
 ('試圖以規劃設計費的名義讓賄款合法化，李並教唆都委會2名承辦人登載不實資料，企圖矇混過關。', '李'),
 ('更三審依違背職務收賄罪，判李、其他人名各12年、9年徒刑，其他人名以行賄罪判刑1年9月。', '李')]

In [4]:
def create_dataset(sentance_list:list) -> pd.DataFrame:
    AML = pd.DataFrame(sentance_list, columns=['sentance', 'name'])
    name_list = []
    full_name = [s[1] for s in sentance_list]
    full_3name = [s[1] for s in sentance_list if len(s[1]) == 3]
    a = Counter([name[0] for name in full_3name])
    keep = [k for k,v in a.items() if v == 1]
    full_3name_filter = [name for name in full_3name if name[0] in keep]
    name_dict = dict((name[0], name) for name in full_3name_filter)   # ex: {'陳' : '陳水扁'}
    name_dict_2 = dict(zip([name[0:2] for name in full_3name], full_3name))  # ex: {'王音': '王音之'}
    for name in full_name:
        if (name[0] in name_dict.keys()) & (len(name) == 1):
            name_list.append(name_dict.get(name[0]))
        elif (name[0] in name_dict.keys()) & (len(name) == 2) & (name[-1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                              '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                              '和', '揆', '要', '再', '董', '涉', '母',\
                                                                              '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                              '趁', '仔', '依', '氏', '父']):
            name_list.append(name_dict.get(name[0]))
        elif (name in name_dict_2.keys()) & (len(name) == 2):
            name_list.append(name_dict_2.get(name))
        else:
            name_list.append(name)
    # 排除重複資料、排除一字、兩字簡稱、兩字三字四字姓不在姓氏表中的人
    AML['name'] = name_list
    AML = AML.drop_duplicates()
    AML = AML[AML['name'].apply(lambda x: (len(x) > 1) )]
    AML = AML[~AML['name'].apply(lambda x: (len(x) == 2) & (x[1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                     '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                     '和', '揆', '要', '再', '董', '涉', '母',\
                                                                     '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                     '趁', '仔', '依', '氏', '父']))]
    return AML



In [5]:
AML = create_dataset(sentance_list)

In [6]:
AML

,sentance,name
0,業者其他人名為讓計畫案順利進行，以250萬元行賄時任內政部都市計畫委員會委員其他人名與其學生...,藍秀琪
1,以250萬元行賄時任內政部都市計畫委員會委員其他人名與其學生藍秀琪；經法院12年審理，201...,藍秀琪
2,其他人名在西華當面向其他人名提議，可利用他的碩士在職專班學生藍秀琪當「白手套」，由藍秀琪開設...,藍秀琪
3,可利用他的碩士在職專班學生藍秀琪當「白手套」，由藍秀琪開設的公司來負責規劃開發計畫書，李向王...,藍秀琪
4,12年前爆發的花蓮縣壽豐鄉鯉魚潭風景區的「綠湖國際大飯店」開發弊案 ，業者王桂霜為讓計畫案順...,王桂霜
5,皆褫奪公權6年，王桂霜1年9月、褫奪公權1年；最高法院日前駁回上訴，全案全部定讞。,王桂霜
6,1999年起死回生，業者雙聯投資實業公司董事長王桂霜，申請將鯉魚潭的22公頃公園等用地，,王桂霜
7,王桂霜為求解套，透過花蓮縣政府承辦技士居中介紹，,王桂霜
8,其他人名在西華當面向王桂霜提議，可利用他的碩士在職專班學生其他人名當「白手套」，,王桂霜
9,判李、藍各12年、9年徒刑，王桂霜以行賄罪判刑1年9月。,王桂霜
